In [1]:
import pandas as pd
from geopy.distance import geodesic
import unicodedata

In [2]:
# Définition des chemins des fichiers CSV
file_events = "event_cleaned_10000.csv"
file_venues = "venues_cleaned.csv"
file_venues_addresses = "venue_+_address_joined.csv"

# Charger les fichiers CSV
df_events = pd.read_csv(file_events, encoding="utf-8")
df_venues = pd.read_csv(file_venues, encoding="utf-8")
df_venues_addresses = pd.read_csv(file_venues_addresses, encoding="utf-8")

# Aperçu des fichiers chargés
print(" Aperçu de df_events :")
print(df_events.head())

print("\n Aperçu de df_venues :")
print(df_venues.head())

print("\n Aperçu de df_venues_addresses :")
print(df_venues_addresses.head())

 Aperçu de df_events :
                                                name  \
0                             \tSam Divine in London   
1                   Adam Hopkins: Sorry, What? (WIP)   
2   AC13 & REAPER Present: Alpha Omega [Live Visual]   
3                        ALTERIA @Legend Club Milano   
4            Billige Gala - Musik, Talk & Feminismus   

                                         description  \
0  Powerhouse & internationally renowned DJ, Sam ...   
1  Adam Hopkins' new work in progress show about ...   
2  ALPHA OMEGA: When Titans Collide – AC13 & REAP...   
3  🎸 **ALTERIA** - "Nel Fiore dei tuoi Danni" LIV...   
4  dq agency & rausgegangen präsentieren: Die Bil...   

                               venue_id     local_start_date  \
0  44842457-b062-4d8f-af32-86d27b78ab26  2025-04-05 20:00:00   
1  9d201ec9-968f-4288-b691-ff68e96995cd  2025-03-06 21:45:00   
2  63875860-f2f7-4a95-833c-89e159025bca  2025-02-21 22:00:00   
3  efbee498-a516-4eff-ba06-cbbe99e0de9a  2025-0

# Nettoyage des tables "venue_+_address_joined.csv", "venues_cleaned.csv" et "event_cleaned_10000.csv"

In [3]:
# Fonction de nettoyage des textes (nom, adresse, ville)
def clean_text(text):
    if isinstance(text, str):
        text = text.strip().lower()  # Supprimer espaces inutiles + mettre en minuscule
        text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')  # Supprimer accents
    return text
#Standarisation des texts 
# Nettoyage des noms et adresses dans les 3 datasets
df_events["name"] = df_events["name"].apply(clean_text)
df_venues["name"] = df_venues["name"].apply(clean_text)
df_venues["address"] = df_venues["address"].apply(clean_text)
df_venues["city"] = df_venues["city"].apply(clean_text)
df_venues_addresses["name"] = df_venues_addresses["name"].apply(clean_text)
df_venues_addresses["address"] = df_venues_addresses["address"].apply(clean_text)

In [4]:
# Conversion des latitudes et longitudes en float
for df in [df_venues, df_venues_addresses]:
    df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
    df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

# Suppression des valeurs aberrantes de latitude/longitude
#On s’assure que les valeurs sont bien entre -90 et 90 pour la latitude, et -180 et 180 pour la longitude. 
#Si une coordonnée est invalide, les calculs de distance avec Geopy seront faux et peuvent donner des résultats absurdes.
#Le code supprime les lignes avec des coordonnées invalides
for df in [df_venues, df_venues_addresses]:
    df = df[(df["latitude"].between(-90, 90)) & (df["longitude"].between(-180, 180))]

# Suppression des lignes où la latitude ou la longitude est null
for df in [df_venues, df_venues_addresses]:
    df.dropna(subset=["latitude", "longitude"], inplace=True)

In [5]:
print(f"✅ Nombre de lignes après nettoyage :")
print(f"df_events : {len(df_events)} lignes")
print(f"df_venues : {len(df_venues)} lignes")
print(f"df_venues_addresses : {len(df_venues_addresses)} lignes")

✅ Nombre de lignes après nettoyage :
df_events : 10000 lignes
df_venues : 9941 lignes
df_venues_addresses : 9941 lignes


# Code algorithme position géographie 

In [6]:
# Renommer 'id' en 'venue_id' pour correspondre à df_events
df_venues_addresses.rename(columns={"id": "venue_id"}, inplace=True)

# Faire la jointure sur 'venue_id'
df_combined = df_events.merge(df_venues_addresses, on="venue_id", how="left")

# Vérifier si la jointure est bien faite
print(" Aperçu du DataFrame fusionné :")
print(df_combined.head())

# Supprimer les lignes où latitude ou longitude est NaN
df_combined = df_combined.dropna(subset=["latitude", "longitude"])
print(" Suppression des NaN terminée !")


 Aperçu du DataFrame fusionné :
                                             name_x  \
0                              sam divine in london   
1                  adam hopkins: sorry, what? (wip)   
2  ac13 & reaper present: alpha omega [live visual]   
3                       alteria @legend club milano   
4           billige gala - musik, talk & feminismus   

                                       description_x  \
0  Powerhouse & internationally renowned DJ, Sam ...   
1  Adam Hopkins' new work in progress show about ...   
2  ALPHA OMEGA: When Titans Collide – AC13 & REAP...   
3  🎸 **ALTERIA** - "Nel Fiore dei tuoi Danni" LIV...   
4  dq agency & rausgegangen präsentieren: Die Bil...   

                               venue_id     local_start_date  \
0  44842457-b062-4d8f-af32-86d27b78ab26  2025-04-05 20:00:00   
1  9d201ec9-968f-4288-b691-ff68e96995cd  2025-03-06 21:45:00   
2  63875860-f2f7-4a95-833c-89e159025bca  2025-02-21 22:00:00   
3  efbee498-a516-4eff-ba06-cbbe99e0de9a  202

In [7]:
print(" Colonnes de df_combined :", df_combined.columns.tolist())

 Colonnes de df_combined : ['name_x', 'description_x', 'venue_id', 'local_start_date', 'local_end_date', 'name_y', 'description_y', 'maximum_capacity', 'company_id', 'address_id', 'is_active', 'created_at', 'deleted_at', 'account_id', 'distributor_id', 'marketplace_url', 'extra', 'image_urls', 'id.1', 'address', 'details', 'city', 'postal_code', 'state', 'state_code', 'country', 'country_code', 'latitude', 'longitude', 'is_active.1', 'created_at.1', 'deleted_at.1', 'managers']


In [8]:
# Définir un seuil de distance (en km) pour considérer un lieu comme un doublon
SEUIL_KM = 0.5  # 500 mètres

In [ ]:
# Liste pour stocker les doublons détectés
doublons = []

# Comparer chaque lieu / événement avec tous les autres
for index1, row1 in df_combined.iterrows():
    lat1, lon1 = row1["latitude"], row1["longitude"]
    
    for index2, row2 in df_combined.iterrows():
        if index1 >= index2:  
            continue  # Éviter les comparaisons en double
        
        lat2, lon2 = row2["latitude"], row2["longitude"]

        # Vérifier que les coordonnées sont bien des nombres
        if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
            continue  # Sauter cette comparaison si une valeur est NaN
        
        # Calculer la distance entre les deux points GPS
        distance = geodesic((lat1, lon1), (lat2, lon2)).km

        # Vérifier si la distance est inférieure au seuil
        if distance < SEUIL_KM:
            doublons.append((row1["name_x"], row2["name_x"], row1["venue_id"], row2["venue_id"], distance))


In [ ]:
# Convertir les résultats en DataFrame pour affichage
df_doublons = pd.DataFrame(doublons, columns=["Nom 1", "Nom 2", "Venue_ID 1", "Venue_ID 2", "Distance (km)"])

# Afficher les résultats
print("\n Doublons détectés :")
print(df_doublons) 